In [2]:
import math
import numpy as np
import plotly.graph_objects as go


In [3]:
def get_training_data(count):
    x1 = np.random.normal(loc = 3, scale = 2, size = count)
    x2 = np.random.normal(loc = -1, scale = 2, size = count)
    y = 3 + x1 + 2*x2 + np.random.normal(loc = 0, scale = math.sqrt(2), size = count)
    
    return (x1,x2,y)

In [8]:
x,y,z = get_training_data(10000)

e = z - 3 - x - 2*y


fig = go.Figure(data=[go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=12,
        color=e,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)])

fig.update_layout(
    scene=dict(
        aspectratio = dict( x=1, y=1, z=0.5 ),
        aspectmode = 'manual',
        xaxis_title='X 1',
        yaxis_title='X 2',
        zaxis_title='Y',
    ),
    showlegend = True,
    title='Generated Data Visualised',
)


fig.write_html('stochastic_data_visualise.html')

fig.show()


In [7]:
batch_size = 100
avg_over = 100

J_theta_old = 0
J_theta_new = 1e5
threshold = 0.00001
eta = 0.001

theta_initial = np.zeros(3)

animation_data = []

global_iterator = 0

total_data=1000000

x1_data,x2_data,y_data = get_training_data(total_data)



while abs(J_theta_new-J_theta_old) > threshold:
    
    
    J_theta_old = J_theta_new
    J_theta_new = 0
    
    
    for _ in range(avg_over):
        J_theta = 0
        m = batch_size
        update_direction = np.zeros(3)
        for i in range(global_iterator, global_iterator + batch_size):
            
            x1 = x1_data[i%total_data]
            x2 = x2_data[i%total_data]
            y = y_data[i%total_data]
            
            h_theta = theta_initial[0] + x1*theta_initial[1] + x2*theta_initial[2]
            J_theta += (y - h_theta)**2
            update_direction += (h_theta - y)*np.array([1,x1,x2])
            
        
        J_theta = J_theta/(2*m)
        global_iterator += batch_size
        global_iterator%=total_data
        animation_data.append((theta_initial[0],theta_initial[1],theta_initial[2],J_theta))

        J_theta_new += J_theta   
        theta_initial = theta_initial - eta * (1/m) * update_direction
        
    J_theta_new/=avg_over
    print(f"J_theta: {J_theta_new}, Difference: {abs(J_theta_new-J_theta_old)}, Parameters:{theta_initial}")
    

J_theta: 11.303349579741145, Difference: 99988.69665042026, Parameters:[0.24682889 0.92642735 0.48096243]
J_theta: 5.1303601523845455, Difference: 6.172989427356599, Parameters:[0.354092   1.21797747 0.9057194 ]
J_theta: 3.270268644365208, Difference: 1.8600915080193374, Parameters:[0.43555621 1.34198389 1.20159669]
J_theta: 2.4863191922433154, Difference: 0.7839494521218926, Parameters:[0.50392387 1.39315379 1.41070441]
J_theta: 2.0968052701412745, Difference: 0.38951392210204094, Parameters:[0.57061356 1.4223226  1.54714207]
J_theta: 1.9654031005503012, Difference: 0.13140216959097328, Parameters:[0.63972154 1.45391585 1.63928335]
J_theta: 1.820393891191742, Difference: 0.1450092093585591, Parameters:[0.70277566 1.46227247 1.70619879]
J_theta: 1.7424869020432874, Difference: 0.07790698914845473, Parameters:[0.76456656 1.46851805 1.75389893]
J_theta: 1.7064900339085034, Difference: 0.03599686813478398, Parameters:[0.82318601 1.46023444 1.78898321]
J_theta: 1.681261742677087, Differenc

In [42]:
import pandas as pd

test_df = pd.read_csv("./Data/q2test.csv")
X1 = np.array(test_df['X_1'])
X2 = np.array(test_df['X_2'])
Y =  np.array(test_df['Y'])

# X1

err = Y - (theta_initial[0] + theta_initial[1]*X1 + theta_initial[2]*X2)

# err
err = np.sum(err**2)

err



19796.305641249975

In [26]:
test_df.head()

,0,1,2
0,X_1,X_2,Y
1,16.678,13.018,45.537
2,6.583,-5.539,-1.17
3,-19.837,6.089,-3.646
4,-8.412,6.11,8.137


In [14]:
import plotly.graph_objects as go

theta_0_vals = [t[0] for t in animation_data]
theta_1_vals = [t[1] for t in animation_data]
theta_2_vals = [t[2] for t in animation_data]
J_val = [t[3] for t in animation_data]

hover_text = [f'Theta 0: {t0}<br>Theta 1: {t1}<br>Theta 2: {t2}<br>Cost: {J:.2f}'
              for t0, t1, t2, J in zip(theta_0_vals, theta_1_vals, theta_2_vals, J_val)]

fig = go.Figure(data=go.Scatter3d(
    x=theta_0_vals, y=theta_1_vals, z=theta_2_vals,
    marker=dict(
        size=4,
        color=J_val,
        colorscale='Viridis',
    ),
    line=dict(
        color='darkblue',
        width=2
    ),
    hovertemplate=hover_text,
    name="Stochastic Gradient Descent Path"
))

fig.update_layout(
    scene=dict(
        aspectratio = dict( x=1, y=1, z=0.7 ),
        aspectmode = 'manual',
        xaxis_title='Theta 0',
        yaxis_title='Theta 1',
        zaxis_title='Theta 2',
    ),
    showlegend = True,
    title='Stochastic Gradient Descent Path',
)

fig.write_html('stochastic_path.html')



In [43]:
batch_sizes = [1,100,10000,1000000]
thresholds = [5e-4,5e-4,5e-5,3e-5]
avg_overs = [1000,1000,10,1]


total_data=1000000
x1_data,x2_data,y_data = get_training_data(total_data)

x1_data = np.hstack((x1_data,x1_data))
x2_data = np.hstack((x2_data,x2_data))
y_data = np.hstack((y_data,y_data))

fig_datas = []

for avg_over,batch_size,threshold in zip(avg_overs,batch_sizes,thresholds):
    
    
    J_theta_old = 0
    J_theta_new = 1e5
    
    avg_over = avg_over
    eta = 0.001

    theta_initial = np.zeros(3)

    animation_data = []

    global_iterator = 0
    
    
    number_of_iterations = 0

    while abs(J_theta_new-J_theta_old) > threshold:
    
        
        J_theta_old = J_theta_new
        J_theta_new = 0
        
        
        for _ in range(avg_over):
            J_theta = 0
            m = batch_size
            update_direction = np.zeros(3)
            
            x1_curr = x1_data[global_iterator:global_iterator+batch_size]
            x2_curr = x2_data[global_iterator:global_iterator+batch_size]   
            y_curr = y_data[global_iterator:global_iterator+batch_size]
            
            x_curr = np.column_stack((np.ones(batch_size),x1_curr,x2_curr))
            h_theta = np.dot(x_curr,theta_initial)
            J_theta = np.sum((y_curr - h_theta)**2)
            update_direction += np.dot((np.dot(x_curr, theta_initial) - y_curr) , x_curr)
            
            # for i in range(global_iterator, global_iterator + batch_size):
                
            #     x1 = x1_data[i%total_data]
            #     x2 = x2_data[i%total_data]
            #     y = y_data[i%total_data]
                
            #     h_theta = theta_initial[0] + x1*theta_initial[1] + x2*theta_initial[2]
            #     J_theta += (y - h_theta)**2
            #     update_direction += (h_theta - y)*np.array([1,x1,x2])
                
            
            J_theta = J_theta/(2*m)
            global_iterator += batch_size
            global_iterator%=total_data
            animation_data.append((theta_initial[0],theta_initial[1],theta_initial[2],J_theta))

            J_theta_new += J_theta   
            theta_initial = theta_initial - eta * (1/m) * update_direction
            
        J_theta_new/=avg_over
        number_of_iterations+=1
        # print(f"{batch_size}| J_theta: {J_theta_new}, Difference: {abs(J_theta_new-J_theta_old)}, Parameters:{theta_initial}")

    print(f"Theta learned: {theta_initial} for batch size: {batch_size}, threshold: {threshold}, avg_over: {avg_over} in {number_of_iterations} iterations")
    
    
    theta_0_vals = [t[0] for t in animation_data]
    theta_1_vals = [t[1] for t in animation_data]
    theta_2_vals = [t[2] for t in animation_data]
    J_val = [t[3] for t in animation_data]

    hover_text = [f'Theta 0: {t0}<br>Theta 1: {t1}<br>Theta 2: {t2}<br>Cost: {J:.2f}'
                for t0, t1, t2, J in zip(theta_0_vals, theta_1_vals, theta_2_vals, J_val)]

    
    scplt = go.Scatter3d(
    x=theta_0_vals, y=theta_1_vals, z=theta_2_vals,
    marker=dict(
        size=4,
        color=J_val,
        colorscale='Viridis',
    ),
    line=dict(
        color='darkblue',
        width=2
    ),
    hovertemplate=hover_text,
    name=f"Batch size:{batch_size}, Threshold:{threshold}"
    )
    fig_datas.append(scplt)
    
fig = go.Figure(data=fig_datas)

fig.update_layout(
    scene=dict(
        aspectratio = dict( x=1, y=1, z=0.7 ),
        aspectmode = 'manual',
        xaxis_title='Theta 0',
        yaxis_title='Theta 1',
        zaxis_title='Theta 2',
    ),
    showlegend = True,
    title='Stochastic Gradient Descent Path',
)

fig.write_html('stochastic_path_all.html')
    
    

Theta learned: [2.94963758 0.99962285 1.99758675] for batch size: 1, threshold: 0.0005, avg_over: 1000 in 115 iterations
Theta learned: [2.68722454 1.06797892 1.97787855] for batch size: 100, threshold: 0.0005, avg_over: 1000 in 8 iterations


KeyboardInterrupt: 